# ETL Pipeline Preparation

Workspace/Sandbox to help develop ETL pipeline to be used in process_data.py.


### Import libraries and load datasets


In [2]:
# import libraries
import numpy as np
import pandas as pd
import seaborn as sns
from sqlalchemy import create_engine

In [3]:
# load datasets
data_directory = "./data/"
message_file = "messages.csv"
categories_file = "categories.csv"
database_output_name = "myDisasterDatabase.db"

messages = pd.read_csv(data_directory + message_file)
categories = pd.read_csv(data_directory + categories_file)


In [4]:
print(messages.shape)
messages.head()

(26248, 4)


,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [5]:
print(categories.shape)
categories.head()

(26248, 2)


,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### Merge datasets


In [7]:
messages.drop_duplicates(subset='id', keep='first', inplace=True)
categories.drop_duplicates(subset='id', keep='first', inplace=True)

In [8]:
# merge datasets
df = messages.merge(categories, on='id', how='left')
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [9]:
df.shape

(26180, 5)

### Split `categories` into separate category columns


In [10]:
df['categories'][1]

'related-1;request-0;offer-0;aid_related-1;medical_help-0;medical_products-0;search_and_rescue-0;security-0;military-0;child_alone-0;water-0;food-0;shelter-0;clothing-0;money-0;missing_people-0;refugees-0;death-0;other_aid-1;infrastructure_related-0;transport-0;buildings-0;electricity-0;tools-0;hospitals-0;shops-0;aid_centers-0;other_infrastructure-0;weather_related-1;floods-0;storm-1;fire-0;earthquake-0;cold-0;other_weather-0;direct_report-0'

In [11]:
cat = df['categories'].str.split(pat=';', expand=True)

In [12]:
colnames = cat.iloc[1,:].str.strip('01-')

In [13]:
cat.columns = colnames

In [14]:
cat['direct_report'].value_counts()

direct_report-0    21116
direct_report-1     5064
Name: direct_report, dtype: int64

In [15]:
cat = cat.replace(r'[a-z_]*-', '', regex=True)

In [16]:
print(df.shape)
print(cat.shape)

(26180, 5)
(26180, 36)


In [17]:
#Drop the original categories column
df = df.drop(['categories'], axis=1)

In [18]:
complete_data = df.merge(cat, left_index=True, right_index=True)

### Save the clean dataset into an sqlite database


In [21]:
engine_name = 'sqlite:///' + database_output_name
engine = create_engine(engine_name)
complete_data.to_sql('messages_table', engine, index=False)